In [1]:
#####--------------------------------------------------------------------#####
##### PREPARATION
#####--------------------------------------------------------------------#####
import pandas as pd
import numpy as np
import pathlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm
from typing import List, Union, Optional, Callable
import pickle
from Bio import AlignIO, SeqIO
from ete3 import Tree, TreeNode
from gctree import CollapsedTree

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import umap
from ete3 import Tree, faces, TreeStyle, NodeStyle, TextFace, SequenceFace, COLOR_SCHEMES, CircleFace
from GCTree_preparation import *
import warnings
import math
from matplotlib.patches import Rectangle
warnings.filterwarnings("ignore")

path_to_storage = "/media/hieunguyen/HNHD01/storage/all_BSimons_datasets"
outdir = "/media/hieunguyen/GSHD_HN01/outdir/sc_bulk_BCR_data_analysis_v0.1"

PROJECT = "220701_etc_biopsies"
path_to_main_output = f"{outdir}/tree_analysis/{PROJECT}"
path_to_01_output = os.path.join(path_to_main_output, "01_output")
path_to_08_output = os.path.join(path_to_main_output, "08_output")
path_to_09_output = os.path.join(path_to_main_output, "09_output")
os.system(f"mkdir -p {path_to_09_output}")

output_type = "mouse_based_output"

path_to_trees = os.path.join(path_to_storage, PROJECT, "GCtrees/v0.2", output_type)

all_tree_folder = [item for item in pathlib.Path(path_to_trees).glob("*") if 
                   os.path.isfile(f"{str(item)}/02_dnapars/gctree.out.inference.1.nk") == True]

all_nk_files = [item for item in pathlib.Path(path_to_trees).glob("*/*/*gctree.out.inference.1.nk")]  
print(f"Number of trees: {len(all_tree_folder)}")   

path_to_metadata = "/media/hieunguyen/HNSD01/src/sc_bulk_BCR_data_analysis/preprocessing/220701_etc_biopsies/metadata.csv"
mid_metadata = pd.read_csv(path_to_metadata, sep =";")
mid_metadata = mid_metadata.drop(['Unnamed: 6', 'Unnamed: 7'], axis = 1)
mid_metadata.columns = ['MID', 'mouse', 'age', 'day', 'population', 'label', 'hex color']

##### Re run the summary analysis of all trees and rendering tree figures
# rerun = True
rerun = False

path_to_04_output = os.path.join(outdir, "VDJ_output", "04_output")
thres = 0.85

clonedf = pd.read_csv(os.path.join(path_to_04_output, "full_clonedf_with_mutation_rate.csv"), index_col= [0])
clonedf = clonedf[clonedf['num_mutation'] != "region_not_covered-skip"]
clonedf = clonedf[clonedf['dataset.name'] == "220701_etc_biopsies"]

maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.csv")

def get_tree_sum_abundance(x):
    mouseid = x.split("_")[0]
    path_to_save_tree_svg = os.path.join(path_to_01_output, mouseid)
    os.system(f"mkdir -p {path_to_save_tree_svg}")

    treeobj = saveTreeobj[x] 

    idmapdf = treeobj.idmapseqdf.copy()
    seqdf = treeobj.seqdf.copy()
    seqdf["population"] = seqdf["MID"].apply(lambda x: mid_metadata[mid_metadata["MID"] == x]["population"].values[0])
    seqdf = seqdf.merge(idmapdf, right_on = "seq", left_on = "seq")
    abd = seqdf.abundance.sum()
    return abd

tqdm.pandas()
# Reload the dictionary from the pickle file
with open(f"{path_to_01_output}/saveTreeobj.pkl", "rb") as f:
    saveTreeobj = pickle.load(f)
if os.path.isfile(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv") == False:
    maindf["clone_sum_abundance"] = maindf["cloneid"].progress_apply(lambda x: get_tree_sum_abundance(x))
    maindf["pct_sum_abundance"] = maindf[["clone_sum_abundance", "mouseID"]].progress_apply(lambda x: x[0]/maindf[maindf["mouseID"] == x[1]]["clone_sum_abundance"].sum(), axis = 1)
    maindf = maindf.sort_values(by=  "pct_sum_abundance", ascending = False)
    maindf.to_csv(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv", index=False)
else:
    print("reading in maindf with added abundance value in each mouse ...")
    maindf = pd.read_csv(f"{path_to_01_output}/tree_summarydf.addedAbundance.csv")
color_path = "./hex_color.csv"

if os.path.isfile(os.path.join(path_to_09_output, "tree_featuredf.csv")) == False:
    tree_featuredf  = pd.DataFrame(data = tree_features, columns = ["feature"])

    for cloneid in tqdm(saveTreeobj.keys()):
        mouseid = cloneid.split("_")[0]
        treeobj = saveTreeobj[cloneid] 
        tmp_treedf = treeobj.summarize_tree_features(sample_name = mouseid, tree_name = cloneid).reset_index()
        tmp_treedf = tmp_treedf.tail(tmp_treedf.shape[0] - 2)
        tmp_treedf.columns = ["feature", cloneid]
        tree_featuredf = tree_featuredf.merge(tmp_treedf, right_on = "feature", left_on = "feature")
    tree_featuredf.to_csv(os.path.join(path_to_09_output, "tree_featuredf.csv"), index = False)
else:
    print("reading in saved result tree featuredf")
    tree_featuredf = pd.read_csv(os.path.join(path_to_09_output, "tree_featuredf.csv"))

/home/hieunguyen/miniconda3/envs/ete3_py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of trees: 7618
reading in maindf with added abundance value in each mouse ...
reading in saved result tree featuredf


In [2]:
tree_featuredf

,feature,m31_IGHV14-3-01_IGHJ1-03_42_1.aln,m43_IGHV3-6-01_IGHJ2-01_42_2.aln,m12_IGHV1-58-01_IGHJ2-01_33_2.aln,m31_IGHV1-59-01_IGHJ3-01_36_5.aln,m39_IGHV1-72-01_IGHJ2-01_39_6.aln,m14_IGHV1-81-01_IGHJ4-01_54_2.aln,m42_IGHV1-59-01_IGHJ1-01_45_1.aln,m30_IGHV7-3-01_IGHJ3-01_48_1.aln,m32_IGHV1-81-01_IGHJ2-02_42_1.aln,...,m43_IGHV1-53-01_IGHJ3-01_36_2.aln,m38_IGHV1-84-01_IGHJ2-01_33_1.aln,m53_IGHV1-72-01_IGHJ2-01_48_3.aln,m32_IGHV1-19-01_IGHJ3-01_39_1.aln,m28_IGHV1-64-01_IGHJ4-01_45_1.aln,m30_IGHV1-64-01_IGHJ4-01_42_1.aln,m30_IGHV3-6-01_IGHJ3-01_24_1.aln,m32_IGHV6-3-01_IGHJ3-01_44_2.aln,m37_IGHV1-26-01_IGHJ3-01_42_5.aln,m37_IGHV1-9-01_IGHJ4-01_36_1.aln
0,n_nodes,3.000000,4.000000,4.000000,4.000000,5.000000,4.00,4.00,3.000000,49.000000,...,28.000000,4.00,16.000000,12.000000,3.000000,3.000000,4.000000,9.000000,3.000000,3.000000
1,n_leaves,1.000000,1.000000,2.000000,2.000000,2.000000,2.00,2.00,1.000000,28.000000,...,16.000000,2.00,10.000000,7.000000,1.000000,1.000000,2.000000,5.000000,1.000000,1.000000
2,p_leaves,0.333333,0.250000,0.500000,0.500000,0.400000,0.50,0.50,0.333333,0.571429,...,0.571429,0.50,0.625000,0.583333,0.333333,0.333333,0.500000,0.555556,0.333333,0.333333
3,n_internals,1.000000,2.000000,1.000000,1.000000,2.000000,1.00,1.00,1.000000,20.000000,...,11.000000,1.00,5.000000,4.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000
4,p_internals,0.333333,0.500000,0.250000,0.250000,0.400000,0.25,0.25,0.333333,0.408163,...,0.392857,0.25,0.312500,0.333333,0.333333,0.333333,0.250000,0.333333,0.333333,0.333333
5,n_ptn,1.000000,2.000000,0.000000,0.000000,1.000000,0.00,0.00,1.000000,9.000000,...,1.000000,0.00,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000,1.000000
6,p_ptn,0.333333,0.500000,0.000000,0.000000,0.200000,0.00,0.00,0.333333,0.183673,...,0.035714,0.00,0.000000,0.000000,0.333333,0.333333,0.000000,0.000000,0.333333,0.333333
7,n_obs,2.000000,3.000000,3.000000,3.000000,3.000000,2.00,2.00,2.000000,44.000000,...,21.000000,2.00,14.000000,9.000000,2.000000,2.000000,3.000000,5.000000,2.000000,2.000000
8,p_obs,0.666667,0.750000,0.750000,0.750000,0.600000,0.50,0.50,0.666667,0.897959,...,0.750000,0.50,0.875000,0.750000,0.666667,0.666667,0.750000,0.555556,0.666667,0.666667
9,n_inf,1.000000,1.000000,1.000000,1.000000,2.000000,2.00,2.00,1.000000,5.000000,...,7.000000,2.00,2.000000,3.000000,1.000000,1.000000,1.000000,4.000000,1.000000,1.000000
